# Llama3 大模型微调实战

Llama3 是 Meta 于2024年4月开放的 Llama 系列的最新模型。基于超过 15T token 训练，相当于 Llama 2 数据集的 7 倍多；支持 8K 长文本，改进的 tokenizer 具有 128K token 的词汇量，可实现更好的性能。Llama3 提供两个版本: 8B版本适合在消费级GPU上高效部署和开发，70B版本则专为大规模AI应用设计。每个版本都包括基础和指令调优两种形式：
* Meta-Llama-3-8b: 8B 基础模型
* Meta-Llama-3-8b-instruct: 8B 基础模型的指令调优版
* Meta-Llama-3-70b: 70B 基础模型
* Meta-Llama-3-70b-instruct: 70B 基础模型的指令调优版

本文将以Llama-3-8B-Instruct为例，为您介绍如何在PAI-DSW中微调Llama3大模型。

## 运行环境要求

* Python环境3.9以上，推荐使用V100(16GB)或更高显存的GPU。

## 准备工作
### 下载模型

In [ ]:
!pip install modelscope==1.12.0 transformers==4.37.0

In [ ]:
from modelscope.hub.snapshot_download import snapshot_download
snapshot_download('LLM-Research/Meta-Llama-3-8B-Instruct', cache_dir='.', revision='master')
# snapshot_download('LLM-Research/Meta-Llama-3-8B', cache_dir='.', revision='master')
# snapshot_download('LLM-Research/Meta-Llama-3-70B-Instruct', cache_dir='.', revision='master')
# snapshot_download('LLM-Research/Meta-Llama-3-70B', cache_dir='.', revision='master')

### 准备数据集

接下来，准备微调示例数据集。本文准备了英文诗歌数据集，来微调 Llama3 大模型，使其提高其生成诗歌的表现能力。

您也可以参考该数据集的格式，根据自己的使用场景，准备所需的数据集。通过微调，能够提高大语言模型在特定任务上的回答准确性。

In [ ]:
!wget https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/tutorial/llm_instruct/en_poetry_train.json -O ../dataset/en_poetry_train.json

## 微调模型

接下来，基于已有的训练脚本`../sft.py`，进行模型的LoRA轻量化训练。在训练结束之后，我们将模型参数进行量化，以便使用更少显存进行推理。

示例使用的参数解释如下，请您根据实际情况进行修改：

- `accelerate launch`命令行工具用于在多GPU中启动和管理深度学习训练脚本。
    - `num_processes` 1：设置并行处理的进程数量为 1，即不进行多进程并行处理。
    - `config_file` `../multi_gpu.yaml`：指定配置文件的路径。
    - `../sft.py`：指定要运行的 Python 脚本的路径。
    
    
脚本`../sft.py`接受的参数：
- `--model_name` `./LLM-Research/Meta-Llama-3-8B-Instruct/`：指定预训练模型的路径。
- `--model_type` `llama`：指定模型的类型，这里是 llama。
- `--train_dataset_name` `chinese_medical_train_sampled.json`：指定训练数据集的路径。
- `--num_train_epochs` 3：设置训练的轮次为 3。
- `--batch_size` 8：设置批处理的大小为 8。
- `--seq_length` 128：设置序列的长度为 128。
- `--learning_rate` 5e-4：设置学习率为 0.0005。
- `--lr_scheduler_type` linear：设置学习率调度器类型为线性。
- `--target_modules` k_proj o_proj q_proj v_proj：指定在微调中需要特别关注的模型模块。
- `--output_dir` `lora_model/`：指定输出目录路径，微调后的模型将被保存在这里。
- `--apply_chat_template`：指定训练时应用聊天模板。
- `--use_peft`：在训练过程中使用参数有效调优PEFT（Parameter-Efficient Fine-Tuning）方法。
- `--load_in_4bit`：指示模型权重载入时使用 4 位精度，减少内存消耗。
- `--peft_lora_r` 32：如果使用了 LoRA（Low-Rank Adaptation）作为参数有效调优方法的一部分，这会指定 LoRA 的秩为 32。
- `--peft_lora_alpha` 32：设置 LoRA 参数的另一部分，alpha 的大小为 32。

当运行`accelerate launch`命令时，它会使用这些参数来启动指定的 Python 脚本，并且根据`multi_gpu.yaml`配置文件中的设置，在计算资源允许的范围内进行训练。

In [ ]:
! accelerate launch --num_processes 1 --config_file ../multi_gpu.yaml ../sft.py \
    --model_name  ./LLM-Research/Meta-Llama-3-8B-Instruct/ \
    --model_type llama \
    --train_dataset_name ../dataset/en_poetry_train.json \
    --num_train_epochs 3 \
    --batch_size 8 \
    --seq_length 128 \
    --learning_rate 5e-4 \
    --lr_scheduler_type linear \
    --target_modules k_proj o_proj q_proj v_proj \
    --output_dir lora_model/ \
    --apply_chat_template \
    --use_peft \
    --load_in_4bit \
    --peft_lora_r 32 \
    --peft_lora_alpha 32

接下来，将LoRA权重与基础模型融合。示例使用的参数解释如下：

* RANK=0：环境变量RANK用于分布式训练中，来表示当前进程在所有进程中的序号。设为0表明它是单进程或者是分布式训练中的主进程。
* python `../convert.py`：执行convert.py脚本，用于权重转换或其他转换工作。
* `--model_name` `./LLM-Research/Meta-Llama-3-8B-Instruct/`：指定基础模型的路径。
* `--model_type` llama：指定模型类型，这里是llama。
* `--output_dir` `trained_model/`：指定转换后的模型和权重应该输出保存的目录。
* `--adapter_dir` `lora_model/`：指定包含LoRA适配器权重的目录。

In [7]:
! RANK=0 python ../convert.py \
    --model_name ./LLM-Research/Meta-Llama-3-8B-Instruct/ \
    --model_type llama \
    --output_dir trained_model/ \
    --adapter_dir lora_model/

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:04<00:00,  1.20s/it]


## 推理模型 

微调好模型之后，我们使用模型进行推理，来验证微调的效果。这里我们让模型写一首关于春天的诗歌，生成的作品效果也很好。

In [ ]:
import torch, transformers

# model_id = "./LLM-Research/Meta-Llama-3-8B-Instruct/"
model_id = "./trained_model/"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

messages = [
    {"role": "user", "content": "Write a poem on a topic 'finetune' "},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])